In [1]:
import os

In [2]:
%pwd

'/Users/maithilipawde/Desktop/ChestCancerDetection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/maithilipawde/Desktop/ChestCancerDetection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [11]:
import os
import zipfile
import gdown
import os
import sys
import logging

logging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"

log_dir = "logs"
log_filepath = os.path.join(log_dir,"running_logs.log")
os.makedirs(log_dir, exist_ok=True)


logging.basicConfig(
    level= logging.INFO,
    format= logging_str,

    handlers=[
        logging.FileHandler(log_filepath),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger("cnnClassifierLogger")
from cnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-03 12:10:59,921: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-03 12:10:59,924: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 12:10:59,924: INFO: common: created directory at: artifacts]
[2024-03-03 12:10:59,925: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-03 12:10:59,925: INFO: 1533896105: Downloading data from https://drive.google.com/file/d/1M7tpPzXv1Zy-5mMzL9e-qWd6GMl3xk-E/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1M7tpPzXv1Zy-5mMzL9e-qWd6GMl3xk-E
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1M7tpPzXv1Zy-5mMzL9e-qWd6GMl3xk-E&confirm=t&uuid=cd37f353-27bb-4553-a297-ef71d961a443
To: /Users/maithilipawde/Desktop/ChestCancerDetection/artifacts/data_ingestion/data.zip
100%|██████████| 77.8M/77.8M [00:21<00:00, 3.68MB/s]


[2024-03-03 12:11:23,003: INFO: 1533896105: Downloaded data from https://drive.google.com/file/d/1M7tpPzXv1Zy-5mMzL9e-qWd6GMl3xk-E/view?usp=sharing into file artifacts/data_ingestion/data.zip]
